### Entpacken und Analyse der Rohdaten

Dieses Notebook dient der Vorbereitung und ersten Analyse der verfügbaren Rohdaten.  
Ziel ist es, die gelieferten Datenarchive (ZIP/TAR) zu entpacken, die enthaltenen HTML- und CSV-Dateien im Data Lake abzulegen und die Datenqualität zu prüfen.

Die Ergebnisse dieses Notebooks bilden die Grundlage für die spätere inhaltliche Verarbeitung in 02_verarbeitung_main.ipynb.

#### 1. Import benötigte Pakete

In [9]:
# Standard
import os # Dateipfaden
import pandas as pd # Tabellenverarbeitung (DataFrames)
from datetime import datetime # Datumsangaben

# Datenprofiling
from ydata_profiling import ProfileReport  

# Dateiverwaltung & Entpacken
from glob import glob # Mehrere Dateien suchen
import tarfile # .tar.gz Dateien entpacken

In [10]:
# Pfade
# Projektverzeichnis 
PROJECT_ROOT = r"D:/DBU/ADSC11 ADS-01/Studienarbeit/newspaper-scraping"

# Input-Pfade
INPUT_PATH = os.path.join(PROJECT_ROOT, "input", "raw") # Rohdaten
DATA_LAKE_PATH = os.path.join(INPUT_PATH, "data-lake") # HTML- und CSV-Dateien
ZIP_PATH = os.path.join(INPUT_PATH, "downloaded_zips") # ZIP-Dateien

# Output-Pfad
OUTPUT_PATH = os.path.join(PROJECT_ROOT, "output") 

#### 2. Archiv entpacken und Dateien speichern

In [11]:
# Funktion: Mit tarfile Rohdaten (Format .tar.gz) einlesen, extrahieren & entpacken
def extract_tar_file(ZIP_PATH, DATA_LAKE_PATH): 
    entpackt = 0 # zählen
    with tarfile.open(ZIP_PATH, "r:gz") as tar:
        for member in tar.getmembers():
            
            # HTML- und CSV-Dateien extrahieren
            if member.name.endswith(".html") or member.name.endswith(".csv"):
                target_path = os.path.join(DATA_LAKE_PATH, os.path.basename(member.name))
                
                # Entpacken in HTML-Datei
                with open(target_path, "wb") as f:
                    f.write(tar.extractfile(member).read())
                entpackt += 1
    return entpackt  

In [12]:
# Entpackung: Mit glob nach .tar.gz-Dateien im ZIP-Ordner suchen und extrahieren
gesamt_entpackt = 0
for zip_path in glob(os.path.join(ZIP_PATH, "*.tar.gz")):
    gesamt_entpackt += extract_tar_file(zip_path, DATA_LAKE_PATH)

# Ergebnis anzeigen
print(f"\n{gesamt_entpackt} neue Dateien entpackt.")


85047 neue Dateien entpackt.


In [13]:
# Verfügbare HTML- und CSV-Dateien im DATA_LAKE_PATH zählen und zurückgeben
html_files = glob(os.path.join(DATA_LAKE_PATH, "*.html"))
csv_files = [f for f in glob(os.path.join(DATA_LAKE_PATH, "*.csv"))]

print("Anzahl CSV-Dateien:", len(csv_files))
print("Anzahl HTML-Dateien:", len(html_files))

Anzahl CSV-Dateien: 1490
Anzahl HTML-Dateien: 83557


#### 3. Datenexploration

##### 3.1 CSV-Dateien (Metadaten)

In [14]:
# CSV-Dateien mit glob finden
csv_files = glob(os.path.join(DATA_LAKE_PATH, "*.csv"))

# Einzelne CSV-Dateien laden und in Liste speichern
dfs = []
for f in csv_files:
    try:
        df = pd.read_csv(f)
        # leeere Dateien überspringen
        if not df.empty:
            df["filename"] = os.path.basename(f)
            dfs.append(df)
    except Exception as e:
        print(f"[Fehler] Datei konnte nicht gelesen werden: {f} ({e})")

# In DataFrame zusammenführen
df_metadaten = pd.concat(dfs, ignore_index=True)

In [15]:
# Überblick df_metadaten
df_metadaten.head()

,Unnamed: 0,name,date,file_name,status,original_url,final_url,encoding,filename
0,0,sz,2021-04-01,data-lake/2021-04-01-sz.html,200,https://www.sueddeutsche.de/,https://www.sueddeutsche.de/,UTF-8,2021-04-01.csv
1,1,zeit,2021-04-01,data-lake/2021-04-01-zeit.html,200,https://www.zeit.de/,https://www.zeit.de/index,UTF-8,2021-04-01.csv
2,2,faz,2021-04-01,data-lake/2021-04-01-faz.html,200,https://www.faz.net/,https://www.faz.net/aktuell/,utf-8,2021-04-01.csv
3,3,heise,2021-04-01,data-lake/2021-04-01-heise.html,200,https://www.heise.de/,https://www.heise.de/,UTF-8,2021-04-01.csv
4,4,golem,2021-04-01,data-lake/2021-04-01-golem.html,200,https://www.golem.de/,https://www.golem.de/sonstiges/zustimmung/ausw...,ISO-8859-1,2021-04-01.csv


In [16]:
# Anzahl Zeilen und Spalten
df_metadaten.shape

(83938, 9)

In [17]:
# Übersicht Daten
# Data profiling 
profile = ProfileReport(df_metadaten)

# ProfileReport als html
profile.to_file(os.path.join(OUTPUT_PATH, "rohdaten_profile_report.html"))

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 9/9 [00:03<00:00,  2.36it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

In [18]:
# Zeitraum 
df_metadaten["date"] = pd.to_datetime(df_metadaten["date"])  

print("Zeitraum:")
print("Von:", df_metadaten["date"].min().date())
print("Bis:", df_metadaten["date"].max().date())

Zeitraum:
Von: 2021-04-01
Bis: 2025-04-30


In [19]:
# Überblick Medienquelle
df_metadaten["name"].unique()

array(['sz', 'zeit', 'faz', 'heise', 'golem', 'tagesspiegel', 'taz',
       'abendblatt', 'berliner', 'welt', 'dfi', 'ecoute', 'medium',
       'esslinger', 'kdnuggets', 'handelsblatt', 'ntv', 'danielmiessler',
       'pioneer', 'suedwest', 't3n', 'economist ', 'srf', 'wef', 'kindly',
       'stuttgarter', 'atlantic', 'netzpolitik', 'towardsds',
       'uebermedien', 'vulture', 'boerse', 'buchreport', '54books', 'dlf',
       'dw', 'spiegel', 'cnn', 'bbc', 'dhv', 'mm', 'stern', 'tagesschau',
       'zvw', 'ieee', 'anwaltsverein', 'wiwo', 'dav', 'economist',
       'dw-de', 'dw-en', 'zwanzig', 'blick', 'nzz', 'republik', 'ta',
       'watson-ch', 'nau', 'standard', 'kronen', 'kurier', 'kleine',
       'watson-de', 'vice-de'], dtype=object)

In [20]:
# Anzahl Medienquelle
print("Anzahl Medienquellen:", df_metadaten["name"].nunique())

Anzahl Medienquellen: 64


In [21]:
# Anzahl Medienquelle nach Jahr
# Jahr extrahieren
df_metadaten["year"] = df_metadaten["date"].dt.year

# Gruppieren nach nach Medium und Jahr
df_medien_nach_jahr = df_metadaten.groupby("year")["name"].nunique().reset_index()
df_medien_nach_jahr

,year,name
0,2021,49
1,2022,63
2,2023,59
3,2024,59
4,2025,59


In [22]:
# Anzahl Daten pro Medienquelle
df_metadaten["name"].value_counts()

name
sz                1490
zeit              1490
faz               1490
heise             1490
tagesspiegel      1490
                  ... 
dw                 769
ecoute             385
danielmiessler     385
dfi                385
economist          257
Name: count, Length: 64, dtype: int64

In [23]:
# Übersicht Medienquellen und zugehörige URL
df_medienliste = df_metadaten[["name", "original_url"]].drop_duplicates()
df_medienliste

,name,original_url
0,sz,https://www.sueddeutsche.de/
1,zeit,https://www.zeit.de/
2,faz,https://www.faz.net/
3,heise,https://www.heise.de/
4,golem,https://www.golem.de/
...,...,...
18914,kronen,https://www.krone.at/
18915,kurier,https://kurier.at/
18916,kleine,https://www.kleinezeitung.at/
18917,watson-de,https://www.watson.de/


In [24]:
# Medienliste als CSV exportieren
df_medienliste.to_csv(os.path.join(OUTPUT_PATH, "medien_quellen_uebersicht.csv"))

In [25]:
# Fehlende Werte 
df_metadaten.isna().sum()

Unnamed: 0      0
name            0
date            0
file_name       0
status          0
original_url    0
final_url       0
encoding        2
filename        0
year            0
dtype: int64

In [26]:
# Fehlende Werte beim Encoding 
df_metadaten[df_metadaten["encoding"].isna()][["name", "date", "file_name", "original_url"]]

,name,date,file_name,original_url
3260,atlantic,2021-06-06,data-lake/2021-06-06-atlantic.html,https://www.theatlantic.com/
7763,wef,2021-09-06,data-lake/2021-09-06-wef.html,https://www.weforum.org/


In [27]:
# Duplikate
df_metadaten.duplicated().sum()

np.int64(0)

In [28]:
# HTTP-Status
df_metadaten["status"].value_counts()

status
200    82666
403      953
418      278
404       20
503       12
502        3
500        2
522        2
504        1
406        1
Name: count, dtype: int64

##### 3.2 HTML-Dateien

In [29]:
# HTML-Dateien mit glob finden
html_files = glob(os.path.join(DATA_LAKE_PATH, "*.html"))

# Dateinamen extrahieren und in Liste speichern
html_data = [] 
for f in html_files:
    name = os.path.basename(f)
    html_data.append({"filename": name})

# Als DataFrame speichern
df_html = pd.DataFrame(html_data)

In [30]:
# Überblick df_html
df_html.head()

,filename
0,2021-04-01-54books.html
1,2021-04-01-abendblatt.html
2,2021-04-01-anwaltsverein.html
3,2021-04-01-atlantic.html
4,2021-04-01-bbc.html


In [31]:
# HTML-Dateinamen extrahieren und Jahr ableiten
html_data = []
for f in html_files:
    name = os.path.basename(f)
    year = int(name[:4])  # Jahr direkt aus den ersten 4 Zeichen
    html_data.append({"filename": name, "year": year})

df_html = pd.DataFrame(html_data)

In [32]:
# HTML-Dateien pro Jahr zählen
df_html_jahr = df_html["year"].value_counts().sort_index().reset_index()
df_html_jahr

,year,count
0,2021,13197
1,2022,20321
2,2023,21461
3,2024,21571
4,2025,7007


In [33]:
# HTML-Dateien nach Medium
df_html["medium"] = df_html["filename"].str[11:-5]  # alles zwischen Datum und ".html"
df_html["medium"].value_counts().head(10)

medium
faz           1491
sz            1491
zeit          1491
abendblatt    1490
buchreport    1490
dav           1490
berliner      1490
atlantic      1490
medium        1490
mm            1490
Name: count, dtype: int64

In [34]:
# Ergebnis als CSV exportieren
df_html_medium_counts = df_html["medium"].value_counts().reset_index()
df_html_medium_counts.columns = ["medium", "anzahl_html_dateien"]
df_html_medium_counts.to_csv(os.path.join(OUTPUT_PATH, "html_dateien_pro_medium.csv"))